In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
from theano import config

from deepensemble.utils import jacobs

N = 1000

X, y = jacobs(sample_len=N, seed=42)
y = np.array(y[:, np.newaxis], dtype=config.floatX)
X = np.array(X, dtype=config.floatX)
t = np.arange(N)

# Added Noise
s_noise = 0.0001
nu = np.random.randn(N, 1) * s_noise
# nu[np.random.rand(N,1) > 0.95] += 0.2

z = y + nu

n_train = int(N * 0.5)
i_test = N - n_train

y_train = z[0:n_train]
y_test = z[i_test:N]
X_train = X[0:n_train]
X_test = X[i_test:N]
t_train = t[0:n_train]
t_test = t[i_test:N]

fx_train = y[0:n_train]
fx_test = y[i_test:N]

SNR = np.mean(z) / np.std(z)

print('SNR: %g' % SNR)

SNR: 0.255204


In [2]:
from matplotlib.pyplot import *
%matplotlib notebook
 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

plt.style.use('fivethirtyeight')

fig = plt.figure(figsize=(12, 6), dpi=80)
ax = plt.subplot(211)
ax.plot(t_train, y_train, '.', alpha=0.25, label='Observaciones')
ax.plot(t_train, fx_train, '-', lw=3, label='Jacobs')
plt.title('Muestras Entrenamiento')
plt.xlabel('x')
plt.legend(loc='best', numpoints=3)
plt.tight_layout()

ax = plt.subplot(212)
ax.plot(t_test, y_test, '.', alpha=0.25, label='Observaciones')
ax.plot(t_test, fx_test, '-', lw=3, label='Jacobs')
plt.title('Muestras para Pruebas')
plt.xlabel('x')
plt.legend(loc='best', numpoints=3)
plt.tight_layout()

NameError: name 'plt' is not defined

In [ ]:
from deepensemble.utils.utils_functions import ActivationFunctions
from deepensemble.models.sequential import Sequential
from deepensemble.layers.dense import Dense
from deepensemble.layers.recurrent import RecurrentLayer
from deepensemble.models.ensemblemodel import EnsembleModel
from deepensemble.combiner import *
from deepensemble.metrics import *
from deepensemble.utils import *

n_neurons = 12
n_models = 4
lr = 0.005
batch_size = 50
max_epoch = 300

n_ensemble_models = 5

fn_activation1 = ActivationFunctions.tanh
fn_activation2 = ActivationFunctions.tanh

n_features = X_train.shape[1]

n_output = y_train.shape[1]
n_inputs = n_features

n_neurons_model = int(0.75 * (n_output + n_inputs))

s = ITLFunctions.silverman(np.array(y_train)).eval()
print('Silverman: %0.4g' % s)

In [ ]:
# Create Ensemble
ensemble = get_ensemble_model(name='Ensamble',
                                  n_input=n_features, n_output=n_output,
                                  n_ensemble_models=n_ensemble_models, n_neurons_model=n_neurons_model,
                                  fn_activation1=fn_activation1, fn_activation2=fn_activation2,
                                  cost=mse, name_cost="MSE",
                                  params_update={'learning_rate': lr})

metrics_ensemble = FactoryMetrics.get_metric(ensemble)

# Compile
ensemble.compile(fast=True)

# training
metrics = ensemble.fit(X_train, y_train, max_epoch=max_epoch, batch_size=batch_size, early_stop=False)
print("FINISHED!")

# Compute metricstrain
metrics_ensemble.append_metric(metrics)

In [ ]:
# Create Ensemble NCL
ensembleNCL = get_ensembleNCL_model(name='Ensamble NCL',
                                        n_input=n_features, n_output=n_output,
                                        n_ensemble_models=n_ensemble_models, n_neurons_models=n_neurons_model,
                                        fn_activation1=fn_activation1, fn_activation2=fn_activation2,
                                        lamb=0.8, params_update={'learning_rate': lr})

metrics_ensembleNCL = FactoryMetrics.get_metric(ensembleNCL)

# Compile
ensembleNCL.compile(fast=True)
                      
# training
metrics = ensembleNCL.fit(X_train, y_train, max_epoch=max_epoch, batch_size=batch_size, early_stop=False)
print("FINISHED!")

# Compute metricstrain
metrics_ensembleNCL.append_metric(metrics)

In [ ]:
# Create MLP
mlp = get_mlp_model("MLP (%d neuronas)" % (n_neurons_model * n_ensemble_models),
                           n_input=n_features, n_output=n_output,
                           n_neurons=n_neurons_model,
                           fn_activation1=fn_activation1, fn_activation2=fn_activation2,
                           cost=mse, name_cost="MSE", params_update={'learning_rate': lr})

metrics_mlp = FactoryMetrics.get_metric(mlp)

# Compile
mlp.compile(fast=True)
                      
# training
metrics = mlp.fit(X_train, y_train, max_epoch=max_epoch, batch_size=batch_size, early_stop=False)
print("FINISHED!")

# Compute metricstrain
metrics_mlp.append_metric(metrics)

In [ ]:
# Create Ensemble CIP
ensembleCIP = get_ensembleCIP_model(name='Ensamble CIP',
                                    n_input=n_features, n_output=n_output,
                                    n_ensemble_models=n_ensemble_models, n_neurons_models=n_neurons_model,
                                    is_cip_full=False,
                                    fn_activation1=fn_activation1, fn_activation2=fn_activation2,
                                    dist='CS',
                                    beta=0.2, lamb=0.2, s=s,
                                    lsp=1.5, lsm=0.1,
                                    lr=lr,
                                    bias_layer=True, mse_first_epoch=False, annealing_enable=True,
                                    update=sgd, name_update='SGD',
                                    params_update={'learning_rate': -lr})

metrics_ensembleCIP = FactoryMetrics.get_metric(ensembleCIP)

# Compile
ensembleCIP.compile(fast=True)
                      
# training
metrics = ensembleCIP.fit(X_train, y_train, max_epoch=max_epoch, batch_size=batch_size, early_stop=False,
                         criterion_update_params = 'cost', maximization_criterion=True)
print("FINISHED!")

# Compute metricstrain
metrics_ensembleCIP.append_metric(metrics)

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=80)

plt.plot(t, z, 'k.', alpha=0.15,  label='Muestras')
plt.plot(t_train, ensemble.predict(X_train), lw=2, label='Ensamble')
plt.plot(t_train, ensembleNCL.predict(X_train), lw=2, label='Ensamble NCL')
plt.plot(t_train, ensembleCIP.predict(X_train), lw=2, label='Ensamble CIP')
plt.plot(t_train, mlp.predict(X_train), lw=2, label='MLP')
plt.title('Conjunto Entrenamiento Jacobs')
plt.xlabel('x')
plt.legend(loc='best')

fig = plt.figure(figsize=(12, 6), dpi=80)

plt.plot(t, z, 'k.', alpha=0.15, label='Muestras')
plt.plot(t_test, ensemble.predict(X_test), lw=2, label='Ensamble')
plt.plot(t_test, ensembleNCL.predict(X_test), lw=2, label='Ensamble NCL')
plt.plot(t_test, ensembleCIP.predict(X_test), lw=2, label='Ensamble CIP')
plt.plot(t_test, mlp.predict(X_test), lw=2, label='MLP')
plt.title('Conjunto Prueba Jacobs')
plt.xlabel('x')
plt.legend(loc='best')

plt.tight_layout()

In [ ]:
from sklearn.neighbors.kde import KernelDensity

def plot_pdf_error(pred, target, label_plot, ax, fig, n_points=1000, xmin=-3, xmax=3):
    error = pred - target
    N = len(error)
    s = 1.06 * np.std(error) / np.power(N, 0.2)  # Silverman
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
    kde.fit(error)
    x_plot = np.linspace(xmin, xmax, n_points)[:, np.newaxis]
    y_plot = np.exp(kde.score_samples(x_plot))
    ax.plot(x_plot, y_plot / np.sum(y_plot), label=label_plot)
    
fig = plt.figure(figsize=(10, 5), dpi=80)
ax = fig.add_subplot(1, 1, 1)

plot_pdf_error(ensemble.predict(X_train), y_train, 'Ensamble', ax, fig)
plot_pdf_error(ensembleNCL.predict(X_train), y_train, 'Ensamble NCL', ax, fig)
plot_pdf_error(ensembleCIP.predict(X_train), y_train, 'Ensamble CIP', ax, fig)
plot_pdf_error(mlp.predict(X_train), y_train, 'MLP', ax, fig)

plt.xlabel('Error')
plt.ylabel('PDF del error');
plt.title("Función de Probabilidad (pdf) del Error conjunto Entrenamiento")
plt.legend()

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=80)
ax = fig.add_subplot(1, 1, 1)

plot_pdf_error(ensemble.predict(X_test), y_test, 'Ensamble', ax, fig)
plot_pdf_error(ensembleNCL.predict(X_test), y_test, 'Ensamble NCL', ax, fig)
plot_pdf_error(ensembleCIP.predict(X_test), y_test, 'Ensamble CIP', ax, fig)
plot_pdf_error(mlp.predict(X_test), y_test, 'MLP', ax, fig)

plt.xlabel('Error')
plt.ylabel('PDF del error');
plt.title("Función de Probabilidad (pdf) del Error\n conjunto Entrenamiento")
plt.legend()

plt.tight_layout()

plt.show()

In [ ]:
score_test_ensemble = ensemble.score(X_test, y_test)
score_train_ensemble = ensemble.score(X_train, y_train)

score_test_ensembleNCL = ensembleNCL.score(X_test, y_test)
score_train_ensembleNCL = ensembleNCL.score(X_train, y_train)

score_test_ensembleCIP = ensembleCIP.score(X_test, y_test)
score_train_ensembleCIP = ensembleCIP.score(X_train, y_train)

score_test_mlp = mlp.score(X_test, y_test)
score_train_mlp = mlp.score(X_train, y_train)

print('Score RMS')
print('Ensamble: %f / %f' % (score_train_ensemble, score_test_ensemble))
print('Ensamble NCL: %f / %f' % (score_train_ensembleNCL, score_test_ensembleNCL))
print('Ensamble CIP: %f / %f' % (score_train_ensembleCIP, score_test_ensembleCIP))
print('MLP: %f / %f' % (score_train_mlp, score_test_mlp))